# Http Requests

## Usage

To construct a HTTP request packet you have a variety of facilities available.

Previously we saw how to parse HTTP responses using `HttpParser`.  We also saw how `HttpParser` class is capable of parsing various type of HTTP protocols.  Remember the _take away_ from that tutorial.

In [1]:
from proxy.http.parser import HttpParser, httpParserTypes
from proxy.http import httpMethods
from proxy.common.utils import HTTP_1_1

request = HttpParser(httpParserTypes.REQUEST_PARSER)
request.path, request.method, request.version = b'/', httpMethods.GET, HTTP_1_1
request.add_header(b'Host', b'jaxl.com')

print(request.build())

b'GET / HTTP/1.1\r\nHost: jaxl.com\r\nUser-Agent: proxy.py v2.4.0rc9.dev8+gea0253d.d20220126\r\n\r\n'


But, this is a painful way to construct request packets.  Hence, other high level abstractions are available.

Example, following one liner will give us the same request packet.

In [2]:
from proxy.common.utils import build_http_request

build_http_request(
    method=httpMethods.GET,
    url=b'/',
    headers={b'Host': b'jaxl.com'},
)

b'GET / HTTP/1.1\r\nHost: jaxl.com\r\nUser-Agent: proxy.py v2.4.0rc9.dev8+gea0253d.d20220126\r\n\r\n'

`build_http_request` ensures a `User-Agent` header.  You can provide your own too:

In [3]:
build_http_request(
    method=httpMethods.GET,
    url=b'/',
    headers={
        b'Host': b'jaxl.com',
        b'User-Agent': b'my app v1'
    },
)

b'GET / HTTP/1.1\r\nHost: jaxl.com\r\nUser-Agent: my app v1\r\n\r\n'

Or, if you don't want a `User-Agent` header at all

In [4]:
build_http_request(
    method=httpMethods.GET,
    url=b'/',
    headers={b'Host': b'jaxl.com'},
    no_ua=True,
)

b'GET / HTTP/1.1\r\nHost: jaxl.com\r\n\r\n'

To add a connection close header, simply pass `conn_close=True`

In [5]:
build_http_request(
    method=httpMethods.GET,
    url=b'/',
    headers={b'Host': b'jaxl.com'},
    conn_close=True,
)

b'GET / HTTP/1.1\r\nHost: jaxl.com\r\nUser-Agent: proxy.py v2.4.0rc9.dev8+gea0253d.d20220126\r\nConnection: close\r\n\r\n'

For `POST` requests, provide the `body` attribute

In [6]:
build_http_request(
    method=httpMethods.POST,
    url=b'/',
    headers={b'Host': b'jaxl.com'},
    body=b'key=value&hello=world',
    content_type=b'application/x-www-form-urlencoded',
    conn_close=True,
)

b'POST / HTTP/1.1\r\nHost: jaxl.com\r\nContent-Type: application/x-www-form-urlencoded\r\nContent-Length: 21\r\nUser-Agent: proxy.py v2.4.0rc9.dev8+gea0253d.d20220126\r\nConnection: close\r\n\r\nkey=value&hello=world'

For chunked data, simply include a `Transfer-Encoding` header.  This will omit the `Content-Length` header then:

In [9]:
from proxy.http.parser import ChunkParser

build_http_request(
    method=httpMethods.POST,
    url=b'/',
    headers={
        b'Host': b'jaxl.com',
        b'Transfer-Encoding': b'chunked',
    },
    body=ChunkParser.to_chunks(b'key=value&hello=world', chunk_size=5),
    content_type=b'application/x-www-form-urlencoded',
    conn_close=True,
)

b'POST / HTTP/1.1\r\nHost: jaxl.com\r\nTransfer-Encoding: chunked\r\nContent-Type: application/x-www-form-urlencoded\r\nUser-Agent: proxy.py v2.4.0rc9.dev8+gea0253d.d20220126\r\nConnection: close\r\n\r\n5\r\nkey=v\r\n5\r\nalue&\r\n5\r\nhello\r\n5\r\n=worl\r\n1\r\nd\r\n0\r\n\r\n'